# PySpark

![Logo](images/apache_spark_logo.png)

- [Apache Spark](https://spark.apache.org) was first released in 2014. 
- It was originally developed by [Matei Zaharia](http://people.csail.mit.edu/matei) as a class project, and later a PhD dissertation, at University of California, Berkeley.
- Spark is written in [Scala](https://www.scala-lang.org).
- All images come from [Databricks](https://databricks.com/product/getting-started-guide).

- Apache Spark is a fast and general-purpose cluster computing system. 
- It provides high-level APIs in Java, Scala, Python and R, and an optimized engine that supports general execution graphs.
- Spark can manage "big data" collections with a small set of high-level primitives like `map`, `filter`, `groupby`, and `join`.  With these common patterns we can often handle computations that are more complex than map, but are still structured.
- It also supports a rich set of higher-level tools including [Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html) for SQL and structured data processing, [MLlib](https://spark.apache.org/docs/latest/ml-guide.html) for machine learning, [GraphX](https://spark.apache.org/docs/latest/graphx-programming-guide.html) for graph processing, and Spark Streaming.

## Resilient distributed datasets

- The fundamental abstraction of Apache Spark is a read-only, parallel, distributed, fault-tolerent collection called a resilient distributed datasets (RDD).
- RDDs behave a bit like Python collections (e.g. lists).
- When working with Apache Spark we iteratively apply functions to every item of these collections in parallel to produce *new* RDDs.
- The data is distributed across nodes in a cluster of computers.
- Functions implemented in Spark can work in parallel across elements of the collection.
- The  Spark framework allocates data and processing to different nodes, without any intervention from the programmer.
- RDDs automatically rebuilt on machine failure.

## Lifecycle of a Spark Program

1. Create some input RDDs from external data or parallelize a collection in your driver program.
2. Lazily transform them to define new RDDs using transformations like `filter()` or `map()`
3. Ask Spark to cache() any intermediate RDDs that will need to be reused.
4. Launch actions such as count() and collect() to kick off a parallel computation, which is then optimized and executed by Spark.

## Operations on Distributed Data

- Two types of operations: **transformations** and **actions**
- Transformations are *lazy* (not computed immediately) 
- Transformations are executed when an action is run

## [Transformations](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations) (lazy)

```
map() flatMap()
filter() 
mapPartitions() mapPartitionsWithIndex() 
sample()
union() intersection() distinct()
groupBy() groupByKey()
reduceBy() reduceByKey()
sortBy() sortByKey()
join()
cogroup()
cartesian()
pipe()
coalesce()
repartition()
partitionBy()
...
```

## [Actions](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions)

```
reduce()
collect()
count()
first()
take()
takeSample()
saveToCassandra()
takeOrdered()
saveAsTextFile()
saveAsSequenceFile()
saveAsObjectFile()
countByKey()
foreach()
```

## Python API

PySpark uses Py4J that enables Python programs to dynamically access Java objects.

![PySpark Internals](images/YlI8AqEl.png)

## The `SparkContext` class

- When working with Apache Spark we invoke methods on an object which is an instance of the `pyspark.SparkContext` context.

- Typically, an instance of this object will be created automatically for you and assigned to the variable `sc`.

- The `parallelize` method in `SparkContext` can be used to turn any ordinary Python collection into an RDD;
    - normally we would create an RDD from a large file or an HBase table.

## First example

PySpark isn't on sys.path by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding pyspark to sys.path at runtime. [findspark](https://github.com/minrk/findspark) does the latter.

We have a spark context sc to use with a tiny local spark cluster with 4 nodes (will work just fine on a multicore machine).

In [1]:
import os, sys
sys.executable

'/opt/conda/envs/big-data/bin/python'

In [2]:
#os.environ["SPARK_HOME"] = "/opt/spark-3.0.1-bin-hadoop2.7"
os.environ["PYSPARK_PYTHON"] = sys.executable

In [3]:
import pyspark

sc = pyspark.SparkContext(master="local[*]", appName="FirstExample")
sc.setLogLevel("ERROR")

In [4]:
print(sc) # it is like a Pool Processor executor

<SparkContext master=local[*] appName=FirstExample>


## Create your first RDD

In [5]:
data = list(range(8))
rdd = sc.parallelize(data) # create collection
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

### Exercise

Create a file `sample.txt`with lorem package. Read and load it into a RDD with the `textFile` spark function.

In [6]:
from faker import Faker
fake = Faker()
Faker.seed(0)

with open("sample.txt","w") as f:
    f.write(fake.text(max_nb_chars=1000))
    
rdd = sc.textFile("sample.txt")

### Collect

Action / To Driver: Return all items in the RDD to the driver in a single list

![](images/DUO6ygB.png)

Source: https://i.imgur.com/DUO6ygB.png

### Exercise 

Collect the text you read before from the `sample.txt`file.

### Map

Transformation / Narrow: Return a new RDD by applying a function to each element of this RDD

![](images/PxNJf0U.png)

Source: http://i.imgur.com/PxNJf0U.png

In [7]:
rdd = sc.parallelize(list(range(8)))
rdd.map(lambda x: x ** 2).collect() # Square each element

[0, 1, 4, 9, 16, 25, 36, 49]

### Exercise

Replace the lambda function by a function that contains a pause (sleep(1)) and check if the `map` operation is parallelized.

### Filter

Transformation / Narrow: Return a new RDD containing only the elements that satisfy a predicate

![](images/GFyji4U.png)
Source: http://i.imgur.com/GFyji4U.png

In [8]:
# Select only the even elements
rdd.filter(lambda x: x % 2 == 0).collect()

[0, 2, 4, 6]

### FlatMap

Transformation / Narrow: Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results

![](images/TsSUex8.png)

In [9]:
rdd = sc.parallelize([1,2,3])
rdd.flatMap(lambda x: (x, x*100, 42)).collect()

[1, 100, 42, 2, 200, 42, 3, 300, 42]

### Exercise

Use FlatMap to clean the text from `sample.txt`file. Lower, remove dots and split into words.

### GroupBy

Transformation / Wide: Group the data in the original RDD. Create pairs where the key is the output of a user function, and the value is all items for which the function yields this key.

![](images/gdj0Ey8.png)

In [10]:
rdd = sc.parallelize(['John', 'Fred', 'Anna', 'James'])
rdd = rdd.groupBy(lambda w: w[0])
[(k, list(v)) for (k, v) in rdd.collect()]

[('J', ['John', 'James']), ('F', ['Fred']), ('A', ['Anna'])]

### GroupByKey

Transformation / Wide: Group the values for each key in the original RDD. Create a new pair where the original key corresponds to this collected group of values.

![](images/TlWRGr2.png)

In [11]:
rdd = sc.parallelize([('B',5),('B',4),('A',3),('A',2),('A',1)])
rdd = rdd.groupByKey()
[(j[0], list(j[1])) for j in rdd.collect()]

[('B', [5, 4]), ('A', [3, 2, 1])]

### Join

Transformation / Wide: Return a new RDD containing all pairs of elements having the same key in the original RDDs

![](images/YXL42Nl.png)

In [12]:
x = sc.parallelize([("a", 1), ("b", 2)])
y = sc.parallelize([("a", 3), ("a", 4), ("b", 5)])
x.join(y).collect()

[('a', (1, 3)), ('a', (1, 4)), ('b', (2, 5))]

### Distinct

Transformation / Wide: Return a new RDD containing distinct items from the original RDD (omitting all duplicates)

![](images/Vqgy2a4.png)

In [13]:
rdd = sc.parallelize([1,2,3,3,4])
rdd.distinct().collect()

[4, 1, 2, 3]

### KeyBy

Transformation / Narrow: Create a Pair RDD, forming one pair for each item in the original RDD. The pair’s key is calculated from the value via a user-supplied function.

![](images/nqYhDW5.png)

In [14]:
rdd = sc.parallelize(['John', 'Fred', 'Anna', 'James'])
rdd.keyBy(lambda w: w[0]).collect()

[('J', 'John'), ('F', 'Fred'), ('A', 'Anna'), ('J', 'James')]

## Actions

### Map-Reduce operation 

Action / To Driver: Aggregate all the elements of the RDD by applying a user function pairwise to elements and partial results, and return a result to the driver

![](images/R72uzwX.png)

In [15]:
from operator import add
rdd = sc.parallelize(list(range(8)))
rdd.map(lambda x: x ** 2).reduce(add) # reduce is an action!

140

### Max, Min, Sum, Mean, Variance, Stdev

Action / To Driver: Compute the respective function (maximum value, minimum value, sum, mean, variance, or standard deviation) from a numeric RDD

![](images/HUCtib1.png)

### CountByKey

Action / To Driver: Return a map of keys and counts of their occurrences in the RDD

![](images/jvQTGv6.png)

In [16]:
rdd = sc.parallelize([('J', 'James'), ('F','Fred'), 
                    ('A','Anna'), ('J','John')])

rdd.countByKey()

defaultdict(int, {'J': 2, 'F': 1, 'A': 1})

In [17]:
# Stop the local spark cluster
sc.stop()

### Exercise 10.1 Word-count in Apache Spark

- Write the sample text file

In [18]:
from lorem import text
with open('sample.txt','w') as f:
    f.write(text())

- Create the rdd with `SparkContext.textFile method`
- lower, remove dots and split using `rdd.flatMap`
- use `rdd.map` to create the list of key/value pair (word, 1)
- `rdd.reduceByKey` to get all occurences
- `rdd.takeOrdered`to get sorted frequencies of words

All documentation is available [here](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html?highlight=textfile#pyspark.SparkContext) for textFile and [here](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html?highlight=textfile#pyspark.RDD) for RDD. 

For a global overview see the Transformations section of the [programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)


## SparkSession

Since SPARK 2.0.0,  SparkSession provides a single point 
of entry to interact with Spark functionality and
allows programming Spark with DataFrame and Dataset APIs. 

###  $\pi$ computation example

- We can estimate an approximate value for $\pi$ using the following Monte-Carlo method:

1.    Inscribe a circle in a square
2.    Randomly generate points in the square
3.    Determine the number of points in the square that are also in the circle
4.    Let $r$ be the number of points in the circle divided by the number of points in the square, then $\pi \approx 4 r$.
    
- Note that the more points generated, the better the approximation

See [this tutorial](https://computing.llnl.gov/tutorials/parallel_comp/#ExamplesPI).


### Exercise 9.2

Using the same method than the PI computation example, compute the integral
$$
I = \int_0^1 \exp(-x^2) dx
$$
You can check your result with numpy

In [19]:
# numpy evaluates solution using numeric computation. 
# It uses discrete values of the function
import numpy as np
x = np.linspace(0,1,1000)
np.trapz(np.exp(-x*x),x)

0.7468240713763741

numpy and scipy evaluates solution using numeric computation. It uses discrete values of the function

In [20]:
import numpy as np
from scipy.integrate import quad
quad(lambda x: np.exp(-x*x), 0, 1)
# note: the solution returned is complex 

(0.7468241328124271, 8.291413475940725e-15)

### Correlation between daily stock

- Data preparation

In [21]:
import os  # library to get directory and file paths
import tarfile # this module makes possible to read and write tar archives

def extract_data(name, where):
    datadir = os.path.join(where,name)
    if not os.path.exists(datadir):
       print("Extracting data...")
       tar_path = os.path.join(where, name+'.tgz')
       with tarfile.open(tar_path, mode='r:gz') as data:
          data.extractall(where)
            
extract_data('daily-stock','data') # this function call will extract json files

Extracting data...


In [22]:
import json
import pandas as pd
import os, glob

here = os.getcwd()
datadir = os.path.join(here,'data','daily-stock')
filenames = sorted(glob.glob(os.path.join(datadir, '*.json')))
filenames

['/home/jovyan/notebooks/data/daily-stock/aet.json',
 '/home/jovyan/notebooks/data/daily-stock/afl.json',
 '/home/jovyan/notebooks/data/daily-stock/aig.json',
 '/home/jovyan/notebooks/data/daily-stock/al.json',
 '/home/jovyan/notebooks/data/daily-stock/amgn.json',
 '/home/jovyan/notebooks/data/daily-stock/avy.json',
 '/home/jovyan/notebooks/data/daily-stock/b.json',
 '/home/jovyan/notebooks/data/daily-stock/bwa.json',
 '/home/jovyan/notebooks/data/daily-stock/ge.json',
 '/home/jovyan/notebooks/data/daily-stock/hal.json',
 '/home/jovyan/notebooks/data/daily-stock/hp.json',
 '/home/jovyan/notebooks/data/daily-stock/hpq.json',
 '/home/jovyan/notebooks/data/daily-stock/ibm.json',
 '/home/jovyan/notebooks/data/daily-stock/jbl.json',
 '/home/jovyan/notebooks/data/daily-stock/jpm.json',
 '/home/jovyan/notebooks/data/daily-stock/luv.json',
 '/home/jovyan/notebooks/data/daily-stock/met.json',
 '/home/jovyan/notebooks/data/daily-stock/pcg.json',
 '/home/jovyan/notebooks/data/daily-stock/tgt.json

In [23]:
%rm data/daily-stock/*.h5

rm: cannot remove 'data/daily-stock/*.h5': No such file or directory


In [24]:
from glob import glob
import os, json
import pandas as pd

for fn in filenames:
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    
    out_filename = fn[:-5] + '.h5'
    df.to_hdf(out_filename, '/data')
    print("Finished : %s" % out_filename.split(os.path.sep)[-1])

filenames = sorted(glob(os.path.join('data', 'daily-stock', '*.h5')))  # data/json/*.json

Finished : aet.h5
Finished : afl.h5
Finished : aig.h5
Finished : al.h5
Finished : amgn.h5
Finished : avy.h5
Finished : b.h5
Finished : bwa.h5
Finished : ge.h5
Finished : hal.h5
Finished : hp.h5
Finished : hpq.h5
Finished : ibm.h5
Finished : jbl.h5
Finished : jpm.h5
Finished : luv.h5
Finished : met.h5
Finished : pcg.h5
Finished : tgt.h5
Finished : usb.h5
Finished : xom.h5


### Sequential code

In [25]:
filenames

['data/daily-stock/aet.h5',
 'data/daily-stock/afl.h5',
 'data/daily-stock/aig.h5',
 'data/daily-stock/al.h5',
 'data/daily-stock/amgn.h5',
 'data/daily-stock/avy.h5',
 'data/daily-stock/b.h5',
 'data/daily-stock/bwa.h5',
 'data/daily-stock/ge.h5',
 'data/daily-stock/hal.h5',
 'data/daily-stock/hp.h5',
 'data/daily-stock/hpq.h5',
 'data/daily-stock/ibm.h5',
 'data/daily-stock/jbl.h5',
 'data/daily-stock/jpm.h5',
 'data/daily-stock/luv.h5',
 'data/daily-stock/met.h5',
 'data/daily-stock/pcg.h5',
 'data/daily-stock/tgt.h5',
 'data/daily-stock/usb.h5',
 'data/daily-stock/xom.h5']

In [26]:
with pd.HDFStore('data/daily-stock/aet.h5') as hdf:
    # This prints a list of all group names:
    print(hdf.keys())

['/data']


In [27]:
df_test = pd.read_hdf('data/daily-stock/aet.h5')

In [28]:
%%time

series = []
for fn in filenames:   # Simple map over filenames
    series.append(pd.read_hdf(fn)["close"])

results = []

for a in series:    # Doubly nested loop over the same collection
    for b in series:  
        if not (a == b).all():     # Filter out comparisons of the same series 
            results.append(a.corr(b))  # Apply function

result = max(results)
result

CPU times: user 1.69 s, sys: 286 ms, total: 1.97 s
Wall time: 2.78 s


0.9413176064560878

### Exercise 9.3

Parallelize the code above with Apache Spark.

- Change the filenames because of the Hadoop environment.

In [29]:
import os, glob

here = os.getcwd()
filenames = sorted(glob.glob(os.path.join(here,'data', 'daily-stock', '*.h5')))
filenames

['/home/jovyan/notebooks/data/daily-stock/aet.h5',
 '/home/jovyan/notebooks/data/daily-stock/afl.h5',
 '/home/jovyan/notebooks/data/daily-stock/aig.h5',
 '/home/jovyan/notebooks/data/daily-stock/al.h5',
 '/home/jovyan/notebooks/data/daily-stock/amgn.h5',
 '/home/jovyan/notebooks/data/daily-stock/avy.h5',
 '/home/jovyan/notebooks/data/daily-stock/b.h5',
 '/home/jovyan/notebooks/data/daily-stock/bwa.h5',
 '/home/jovyan/notebooks/data/daily-stock/ge.h5',
 '/home/jovyan/notebooks/data/daily-stock/hal.h5',
 '/home/jovyan/notebooks/data/daily-stock/hp.h5',
 '/home/jovyan/notebooks/data/daily-stock/hpq.h5',
 '/home/jovyan/notebooks/data/daily-stock/ibm.h5',
 '/home/jovyan/notebooks/data/daily-stock/jbl.h5',
 '/home/jovyan/notebooks/data/daily-stock/jpm.h5',
 '/home/jovyan/notebooks/data/daily-stock/luv.h5',
 '/home/jovyan/notebooks/data/daily-stock/met.h5',
 '/home/jovyan/notebooks/data/daily-stock/pcg.h5',
 '/home/jovyan/notebooks/data/daily-stock/tgt.h5',
 '/home/jovyan/notebooks/data/daily

If it is not started don't forget the PySpark context

Computation time is slower because there is a lot of setup, workers creation, there is a lot of communications the correlation function is too small

### Exercise 9.4 Fasta file example

Use a RDD to calculate the GC content of fasta file nucleotide-sample.txt:

$$\frac{G+C}{A+T+G+C}\times100 \% $$

Create a rdd from fasta file genome.txt in data directory and count 'G' and 'C' then divide by the total number of bases.

### Another example

Compute the most frequent sequence with 5 bases.